# Importing data

In [76]:
import pandas as pd

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df["audio_length"] = df.end-df.start
df = df[df.audio_length >= 8]
df = pd.concat([df, df.speaker_info.apply(pd.Series)], axis=1)
gb = df.groupby("Speaker_name").agg(
    {
        "path": "count"
    }
)

filtered_gb = gb[(gb.path >= 200) & (gb.path <= 3000)]
df = df[df.Speaker_name.isin(filtered_gb.index.tolist())]
df.head(3)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,...,norm_words_edited,audio_length,Speaker_role,Speaker_type,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,rFVDr4ghXlQ_10620.07-10632.05.wav,"20 2 2020 - 1. dio, 16. sjednica, 9. saziv [rF...",10620.07,10632.05,"[interpretacija,, inzistiranje, na, tim, dezin...","[0, 1.0, 1.83, 1.96, 2.22, 3.79, 4.01, 4.43, 5...","[interpretacija, inzistiranje, na, tim, dezinf...","[0, 1.0, 1.83, 1.96, 2.22, 3.79, 4.01, 4.43, 5...",ParlaMint-HR_S16.u4568,ParlaMint-HR_S16.u4568,...,NaN,11.98,Regular,MP,HDZ,Klub Hrvatske demokratske zajednice,Coalition,"Tuđman, Miroslav",M,1946
5,NsYt6iW4zeI_7962.04-7975.62.wav,"29 06 2017 - 2. dio, 4. sjednica, 9. saziv [Ns...",7962.04,7975.62,"[je, nekakav, pokazatelj, razvijenosti, naših,...","[0, 0.07, 0.85, 1.78, 2.92, 3.69, 4.62, 5.5, 6...","[je, nekakav, pokazatelj, razvijenosti, naših,...","[0, 0.07, 0.85, 1.78, 2.92, 3.69, 4.62, 5.5, 6...",ParlaMint-HR_S04.u2695,ParlaMint-HR_S04.u2695,...,NaN,13.58,Regular,MP,SDP,Klub Socijaldemokratske partije Hrvatske,Opposition,"Vešligaj, Marko",M,1982
6,LfkFqOr01h4_2136.52-2155.87.wav,"1 2 2019 - 2. dio, 11. sjednica, 9. saziv [Lfk...",2136.52,2155.87,"[Znači,, ovim, konačnim, prijedlogom, zakona,,...","[0, 0.42, 0.89, 1.21, 1.6, 2.04, 2.47, 2.53000...","[znači, ovim, konačnim, prijedlogom, zakona, p...","[0, 0.42, 0.89, 1.21, 1.6, 2.04, 2.47, 2.53000...",ParlaMint-HR_S11.u2482,ParlaMint-HR_S11.u2482,...,NaN,19.35,Regular,MP,HDZ,Klub Hrvatske demokratske zajednice,Coalition,"Sokol, Tomislav",M,1982


In [77]:
# df = df.dropna()
allowed_party_stati = ["Coalition", "Opposition"]

df = df[df.Party_status.isin(allowed_party_stati)]
df.Party_status.unique()

array(['Coalition', 'Opposition'], dtype=object)

In [78]:
pd.set_option("display.max_rows", 300)

df.groupby("Speaker_gender Party_status".split()).agg({
    "path": "count",
})

path
Speaker_gender Party_status       
F              Coalition     18476
               Opposition     6577
M              Coalition     59444
               Opposition    55854

In [79]:
Opposition_speakers = df.Speaker_name[df.Party_status=="Opposition"].unique()
Coalition_speakers = df.Speaker_name[df.Party_status=="Coalition"].unique()

set(Opposition_speakers.tolist()).intersection(set(Coalition_speakers.tolist()))

set()

In [80]:
df.groupby("Speaker_gender Party_status".split()).agg({
    "path": "count",
    "Speaker_name": lambda i: len(set(i)),
    
}).rename(columns={
    "path": "Instance Count",
    "Speaker_name": "Number of unique speakers"
})

Instance Count  Number of unique speakers
Speaker_gender Party_status                                           
F              Coalition              18476                         17
               Opposition              6577                          8
M              Coalition              59444                         59
               Opposition             55854                         43

In [87]:
25 * 2 * 50

2500

In [82]:
males = df[df.Speaker_gender == "M"]


In [83]:
gb = males.groupby(["Party_status", "Speaker_name"]).agg(
    {"path":"count"}
)
print("Original gb shape: ", gb.shape)
newgb = gb[gb.path>=50]
print("New gb shape: ", newgb.shape)

Original gb shape:  (102, 1)
New gb shape:  (102, 1)


In [89]:
newgb.path.min()

210

In [84]:
newgb.groupby("Party_status").count()

,path
Party_status,
Coalition,59
Opposition,43


In [90]:
test_speaker_count = 9
dev_speaker_count = 9
train_speaker_count = 25

train_instances_per_speaker = 50
test_instances_per_speaker = 50
dev_instances_per_speaker = 50

train_speakers = []
dev_speakers = []
test_speakers = []

for status in ["Opposition", "Coalition"]:
    males_with_status = males[males.Party_status == status].groupby("Speaker_name").agg({"path": "count"}).sort_values("path", ascending=False).index.tolist()
    train_speakers.extend(males_with_status[0:train_speaker_count])
    dev_speakers.extend(males_with_status[train_speaker_count:train_speaker_count+dev_speaker_count])
    test_speakers.extend(males_with_status[train_speaker_count+dev_speaker_count:train_speaker_count+dev_speaker_count+test_speaker_count])


In [91]:
len(test_speakers), len(dev_speakers), len(train_speakers)

(18, 18, 50)

In [92]:
train_paths = []
dev_paths = []
test_paths = []

for speaker in train_speakers:
    train_paths.extend(
        df[df.Speaker_name==speaker].path.tolist()[0:train_instances_per_speaker]
    )
for speaker in test_speakers:
    test_paths.extend(
        df[df.Speaker_name==speaker].path.tolist()[0:test_instances_per_speaker]
    )
for speaker in dev_speakers:
    dev_paths.extend(
        df[df.Speaker_name==speaker].path.tolist()[0:dev_instances_per_speaker]
    )

In [93]:
test_df = df[df.path.isin(test_paths)]
dev_df = df[df.path.isin(dev_paths)]
train_df = df[df.path.isin(train_paths)]

In [94]:
files_to_move = []

for cdf in [test_df, dev_df, train_df]:
    for path in cdf.path:
        files_to_move.append(path)

len(files_to_move)

4300

In [95]:
train_df.shape[0], test_df.shape[0], dev_df.shape[0]

(2500, 900, 900)

In [96]:
with open("012_files_to_move.txt", "w") as f:
    for line in files_to_move:
        f.write(line+"\n")


In [97]:
for cdf in [test_df, dev_df, train_df]:
    cdf["path"] = cdf.path.apply(lambda s: "seg."+s)

for cdf in [test_df, dev_df, train_df]:
    cdf["path"] = cdf.path.apply(lambda s: "data_party_status/"+s)

<ipython-input-97-f7415cb6114f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf["path"] = cdf.path.apply(lambda s: "seg."+s)
<ipython-input-97-f7415cb6114f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf["path"] = cdf.path.apply(lambda s: "data_party_status/"+s)


In [98]:
test_df.loc[:, ["path", "Party_status"]].to_csv("012_test.csv", index=False)
dev_df.loc[:, ["path", "Party_status"]].to_csv("012_dev.csv", index=False)
train_df.loc[:, ["path", "Party_status"]].to_csv("012_train.csv", index=False)